In [2]:
import numpy as np
import skimage as sk
from glob import glob
import napari
from cellpose import io, models, train
import os


In [ ]:
img_files = sorted(glob("../Microglia_CellPose_Model/Images/*.tif"))
mask_files = sorted(glob("../Microglia_CellPose_Model/Masks/*.tif"))
imgs = list(map(sk.io.imread, img_files))
masks = list(map(sk.io.imread, mask_files))

In [ ]:
#os.mkdir('../Microglia_CellPose_Model/Training') #add in if directory does not exist yet
save_imgs = '../Microglia_CellPose_Model/NonEmpty_Slices'
save_masks = '../Microglia_CellPose_Model/NonEmpty_Masks'
for i in range(len(imgs)):
    split_img = np.split(imgs[i], imgs[i].shape[0],axis=0)
    split_mask = np.split(masks[i], masks[i].shape[0],axis=0)
    for c in range(len(split_img)):
        if np.max(split_mask[c]) == 0:
            continue
        else:
            labels = sk.measure.label(split_mask[c] > 0)
            sk.io.imsave(os.path.join(save_imgs,"Img_0"+str(i)+"_"+str(c)+".tif"),split_img[c],check_contrast=False)
            sk.io.imsave(os.path.join(save_masks,"Img_0"+str(i)+"_"+str(c)+"_mask.tif"),labels,check_contrast=False)

In [3]:
X = sorted(glob('../Microglia_CellPose_Model/NonEmpty_Slices/*.tif'))
X = list(map(sk.io.imread,X))
Y = sorted(glob('../Microglia_CellPose_Model/NonEmpty_Masks/*.tif'))
Y = list(map(sk.io.imread,Y))
rng = np.random.RandomState(42)
ind = rng.permutation(len(X))
n_val = max(1, int(round(0.15 * len(ind))))
ind_train, ind_val = ind[:-n_val], ind[-n_val:]
X_val, Y_val = [X[i] for i in ind_val]  , [Y[i] for i in ind_val]
X_trn, Y_trn = [X[i] for i in ind_train], [Y[i] for i in ind_train]

In [4]:
print(len(X_val))

22


# Test header

In [3]:
train_dir = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Training/'
test_dir = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Test/'
for i in range(len(X_trn)):
    sk.io.imsave(os.path.join(train_dir,"Img_0"+str(i)+".tif"),X_trn[i],check_contrast=False)
    sk.io.imsave(os.path.join(train_dir,"Img_0"+str(i)+"_mask.tif"),Y_trn[i],check_contrast=False)

for i in range(len(X_val)):
    sk.io.imsave(os.path.join(test_dir,"Img_0"+str(i)+".tif"),X_val[i],check_contrast=False)
    sk.io.imsave(os.path.join(test_dir,"Img_0"+str(i)+"_mask.tif"),Y_val[i],check_contrast=False)

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\ParadaKusz_Lab\\Microglia_CellPose_Model\\Training\\Img_00.tif'

In [ ]:
train_dir = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Training/'
test_dir = 'E:/ParadaKusz_Lab/Microglia_CellPose_Model/Test/'
io.logger_setup()

output = io.load_train_test_data(train_dir, test_dir,
                                mask_filter="_mask", look_one_level_down=False)
images, labels, image_names, test_images, test_labels, image_names_test = output

model = models.CellposeModel(model_type="cyto3",gpu=True)

model_path, train_losses, test_losses = train.train_seg(model.net,
                            train_data=images, train_labels=labels,
                            test_data=test_images, test_labels=test_labels,
                            weight_decay=0.1, learning_rate=1e-5,
                            n_epochs=100, model_name="Test")